In [6]:
#pip installs:

# pip install requests
# pip install beautifulsoup4
# pip install lxml
# pip install rich

#pip install selenium


#in a jupyter console

# The ratio of the community ratings turned to words:

In [7]:
# js file https://www.allsides.com/sites/default/files/js/js_RkhuUtuGis_wrzoAtkhY74JegSNy1LhoCkjSMXavCTI.js

def community_vote(ratio):
    # the value of the ratio goes through this if statement and prints out the return string
    if ratio > 3:
        return "Absolutely Agree"

    elif 2 < ratio <= 3:
        return "Strongly Agree"

    elif 1.5 < ratio <= 2:
        return "Agree"

    elif 1 < ratio <= 1.5:
        return "Somewhat Agree"

    elif 0.67 < ratio < 1:
        return "Somewhat Disagree"

    elif 0.5 < ratio <= 0.67:
        return "Disagree"

    elif 0.33 < ratio <= 0.5:
        return "Strongly Disagree"

    elif ratio <= .33:
        return "Absolutely Disagree"

    elif ratio == 1:
        return "Neutral"

    else:
        return "N/A"

# Code without infinite scroll

In [12]:
from bs4 import BeautifulSoup
import requests
#from communityfeedback import *
from time import sleep
from rich.progress import track
import json

# TODO: Finish progress bar. 
# TODO: For numbers that are over 10,000 add the comma. 
# TODO: For Allsides Bias Rating change 'left-center' --> 'Left Center'. 


def table(full_table):
    print('Web scraper is parsing the table!')
    # only collects data from the first table that shows not the ones that load after the infinite scroll.
    for url in ['https://www.allsides.com/media-bias/ratings?field_featured_bias_rating_value=All&field_news_source_type_tid[1]=1&field_news_source_type_tid[2]=2&field_news_source_type_tid[3]=3&field_news_source_type_tid[4]=4']:
        source = requests.get(url)
        soup = BeautifulSoup(source.content, 'lxml')
        
        main_table = soup.select('tbody tr')
        global count
        count=0
        

        for row in main_table:
            
            f = dict()

            f['NewsSource'] = row.select_one('.source-title').text.strip()
            f['AllSidesBiasRating'] = row.select_one('.views-field-field-bias-image a')['href'].split('/')[-1].replace("-"," ")
            f['NewsMediaInfo'] = 'https://www.allsides.com' + row.select_one('.source-title a')['href']
            f['Agree'] = int(row.select_one('.agree').text)
            f['Disagree'] = int(row.select_one('.disagree').text)
            f['Ratio'] = (f['Agree'] / f['Disagree'])
            f['CommunityFeedback'] = community_vote(f['Ratio'])
            f['Ratio'] = '{:.3f}'.format(f['Ratio'])
            
            count=count+1
              

            full_table.append(f)  # adds it to the empty list

        sleep(3)  # this is due to the ten seconds before request in robots.txt
    return full_table

def website(full_table):
    headers = {'User-Agent': 'Mozilla/5.0'}
    for f in track(full_table, description='Parsing...'):
        # Initialize default values
        f['Bias Rating'] = 'N/A'
        f['Type'] = 'N/A'
        f['Region'] = 'N/A'
        f['Owned By'] = 'N/A'
        f['Established'] = 'N/A'
        f['Website'] = 'N/A'
        f['Twitter'] = 'N/A'
        f['Facebook'] = 'N/A'
        f['Wikipedia'] = 'N/A'

        response = requests.get(f['News Media Info'], headers=headers)
        soup = BeautifulSoup(response.content, 'lxml')

        source_details_container = soup.find('div', class_='info-table_container')
        if not source_details_container:
            print(f"Details not found for {f['News Source']}")
            continue

        info_table_rows = source_details_container.find_all('tr')

        for row in info_table_rows:
            columns = row.find_all('td')
            if len(columns) == 2:
                label = columns[0].text.strip()
                value_content = columns[1]

                # Extract URL if available, otherwise text
                link = value_content.find('a')
                if link and 'href' in link.attrs:
                    value = link['href'] if link['href'].startswith('http') else link.text
                else:
                    value = value_content.text.strip()

                # Map the label to the corresponding field in your dictionary
                if label == 'Bias Rating':
                    f['Bias Rating'] = value
                elif label == 'Type':
                    f['Type'] = value
                elif label == 'Region':
                    f['Region'] = value
                elif label == 'Owner':
                    f['Owned By'] = value
                elif label == 'Established':
                    f['Established'] = value
                elif label == 'Website':
                    f['Website'] = value
                elif label == 'Twitter':
                    f['Twitter'] = value
                elif label == 'Facebook':
                    f['Facebook'] = value
                elif label == 'Wikipedia':
                    f['Wikipedia'] = value

    # Adjust sleep as per necessity and compliance with robots.txt
    sleep(2)
    print('Parsing has finished')
    return full_table


def saving_data(full_table):
    # saves the data into a json file with no lines
    with open('all-sidesTEST.json', 'w') as i:
        yo = json.dump(full_table, i, indent=4)
        print(yo)
        print(count)


def main():
    #global full_table
    full_table = []  # empty list
    full_table = table(full_table)
    #print(full_table)
    full_table = website(full_table)
    saving_data(full_table)

if __name__ == '__main__':
    main()

Web scraper is parsing the table!


Output()

Parsing has finished
None
50


# THE CODE: (infinite scroll)

In [14]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json


import requests


from rich.progress import track
import json

def init_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=options)
    return driver

def scroll_to_bottom(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(3)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def table(full_table):
    print('Web scraper is parsing the table!')

    url = 'https://www.allsides.com/media-bias/ratings?field_featured_bias_rating_value=All&field_news_source_type_tid[1]=1&field_news_source_type_tid[2]=2&field_news_source_type_tid[3]=3&field_news_source_type_tid[4]=4'
    driver = init_driver()
    driver.get(url)

    scroll_to_bottom(driver)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    main_table = soup.select('tbody tr')
    
    global count
    count=0

    for row in main_table:
        f = dict()
        
        try:
            f['News Source'] = row.select_one('.source-title').text.strip()
            f['AllSides Bias Rating'] = row.select_one('.views-field-field-bias-image a')['href'].split('/')[-1].replace("-"," ")
            f['News Media Info'] = 'https://www.allsides.com' + row.select_one('.source-title a')['href']
            f['Agree'] = int(row.select_one('.agree').text)
            f['Disagree'] = int(row.select_one('.disagree').text)
            f['Ratio'] = (f['Agree'] / f['Disagree'])
            f['Community Feedback'] = community_vote(f['Ratio'])
            f['Ratio'] = '{:.3f}'.format(f['Ratio'])
            
            count=count+1
            
            # Uncomment the following line if community_vote function is available
             #f['Community Feedback'] = community_vote(float(f['Ratio']))
        except Exception as e:
            print(f"Error processing row: {e}")
            continue

        full_table.append(f)

    driver.quit()
    return full_table

def website(full_table):
    headers = {'User-Agent': 'Mozilla/5.0'}
    #for f in track(full_table[5], description="parsing..."):
    for f in track(full_table, description='Parsing...'):
        # Initialize default values
        f['Bias Rating'] = 'N/A'
        f['Type'] = 'N/A'
        f['Region'] = 'N/A'
        f['Owned By'] = 'N/A'
        f['Established'] = 'N/A'
        f['Website'] = 'N/A'
        f['Twitter'] = 'N/A'
        f['Facebook'] = 'N/A'
        f['Wikipedia'] = 'N/A'

        response = requests.get(f['News Media Info'], headers=headers)
        soup = BeautifulSoup(response.content, 'lxml')

        source_details_container = soup.find('div', class_='info-table_container')
        if not source_details_container:
            print(f"Details not found for {f['News Source']}")
            continue

        info_table_rows = source_details_container.find_all('tr')

        for row in info_table_rows:
            columns = row.find_all('td')
            if len(columns) == 2:
                label = columns[0].text.strip()
                value_content = columns[1]

                # Extract URL if available, otherwise text
                link = value_content.find('a')
                if link and 'href' in link.attrs:
                    value = link['href'] if link['href'].startswith('http') else link.text
                else:
                    value = value_content.text.strip()

                # Map the label to the corresponding field in your dictionary
                if label == 'Bias Rating':
                    f['Bias Rating'] = value
                elif label == 'Type':
                    f['Type'] = value
                elif label == 'Region':
                    f['Region'] = value
                elif label == 'Owner':
                    f['Owned By'] = value
                elif label == 'Established':
                    f['Established'] = value
                elif label == 'Website':
                    f['Website'] = value
                elif label == 'Twitter':
                    f['Twitter'] = value
                elif label == 'Facebook':
                    f['Facebook'] = value
                elif label == 'Wikipedia':
                    f['Wikipedia'] = value

    # Adjust sleep as per necessity and compliance with robots.txt
    time.sleep(2)
    print('Parsing has finished')
    return full_table


def saving_data(full_table):
    # saves the data into a json file
    with open('all-sidesFINAL.json', 'w') as i:
        json.dump(full_table, i, indent=4)
        print(count)

def main():
    #global full_table
    full_table = []  
    full_table = table(full_table)
    full_table = website(full_table)
    saving_data(full_table)

if __name__ == '__main__':
    main()


Web scraper is parsing the table!
Error processing row: division by zero
Error processing row: division by zero
Error processing row: division by zero
Error processing row: division by zero
Error processing row: division by zero
Error processing row: division by zero
Error processing row: division by zero
Error processing row: division by zero


Output()

Parsing has finished
2278
